Import thư viện

In [ ]:
import numpy as np
from numpy import log, dot, exp, shape
import pandas as pd
import matplotlib.pyplot as plt  # Các bạn có thể thử plot chart bằng plotly
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 1: Implement hàm sigmoid

In [ ]:
# implement in class MyLogisticRegression

# 2: Implement lại Logistic Regression

In [ ]:
# implement in class MyLogisticRegression

def F1_score(y_actual, y_pred):
    TP = np.sum((y_actual == 1) & (y_pred == 1))
    FP = np.sum((y_actual == 0) & (y_pred == 1))
    TN = np.sum((y_actual == 0) & (y_pred == 0))
    FN = np.sum((y_actual == 1) & (y_pred == 0))
    Precision = TP / (TP + FP)
    Recall = TP / (TP + FN)
    F1 = 2*(Precision*Recall)/(Precision + Recall)
    return F1

In [ ]:
class MyLogisticRegression():

    def __init__(self, w_init, learning_rate=0.05, max_iteration=10000, preprocessing='None'):
        """
        Initialize parameters:
            * w_init = (d+1, 1)
        """
        self.learning_rate = learning_rate
        self.max_iteration = max_iteration
        self.w = w_init

    def preprocessing(self, X, y, preprocessing):
        """
        Standardize or Normalize the features then add the column with all elements equal to '1' into features
            * X: features
            * y: output
            * preprocessing: 1 in 'standardization', 'normalization' or 'None'

        """
        self.X = X
        self.y = y
        self.preprocessing = preprocessing

        # Handle errors when x, y is a row vector
        try:
            self.X.shape[1]
        except:
            self.X = self.X.reshape((X.shape[0], 1))

        try:
            self.y.shape[1]
        except:
            self.y = self.y.reshape((y.shape[0], 1))

        if preprocessing == 'Standardization':
            for i in range(self.X.shape[1]):
                self.X[:, i] = self._Standardization(self.X[:, i])

        elif preprocessing == 'Normalization':
            for i in range(self.X.shape[1]):
                self.X[:, i] = self._Normalization(self.X[:, i])
        elif preprocessing == 'None':
            self.X = self.X
            self.y = self.y

        else:
            raise ValueError(
                "Preprocessing should be 'Standardization' or 'Normalization' or 'None'")

        # Create the new X. all elements in first column equal to 1

        ones = np.ones((self.X.shape[0], 1))
        self.X = np.concatenate((ones, self.X), axis=1)

        return self.X, self.y

    def _Standardization(self, X):
        """
        Standardize X as formula:
            Xi =   (Xi  - X_mean)  / standard_deviation(X)
        with Xi is element ith in X 
        """
        return (X - np.mean(X)) / np.std(X)

    def _Normalization(self, X):
        """
        Normalize X as formula: 
            Xi = (Xi - X_min) / (X_max - X_min)
        with Xi is element ith in X 
        """
        return (X - np.min(X)) / (np.max(X) - np.min(X))

    def _sigmoid(self, a):
        """
        Activation function to transform orginal data into (0,1) and keep the original order (probability properties) 
            sigmoid(z)   =  1 / (1  + e ^ (-z))
        """
        return 1/(1 + np.exp(-a))

    def z(self, w):
        """
        Calculate input before going through the activation function.
        """
        return dot(self.X, w)

    def y_hat(self, w):
        """
        Output after going through the activation function
        """
        return self._sigmoid(self.z(w))

    def loss(self, w):
        """
        Calculate loss at point w:

            loss = -1/N   ( y^T*log(y^) + (1-y)^T * log(1 - y^))
        with N is number of observations

        """
        l = - (np.dot(self.y.T, np.log(self.y_hat(w))) +
             np.dot((1-self.y).T, np.log(1-self.y_hat(w)))) / len(self.y)
        return l

    def gradient(self, w):
        """
        Calculate gradient at point w:
            gradient  = X^T (Y^ - Y)
        """
        gradient = np.dot(self.X.T, (self.y_hat(w) - self.y))
        return gradient

    def get(self):
        """
        Fit data with gradient descent
        Return: * first element: final estimated coefficients
                * second element: all estimated coefficients during training phase
                * last element: list of loss each iteration.
        """
        w_history = []
        w_new = self.w
        loss = np.zeros(self.max_iteration, )
        for i in range(self.max_iteration):
            loss_element = self.loss(w_new)
            loss[i] = loss_element
            w_new = w_new - self.learning_rate * self.gradient(w_new)
            w_history.append(w_new)

        return w_new, w_history, loss

    def predict(self, X_test,  threshold=0.5):
        """
        Predict new observations, if probability > threshhold --> 1 , else ---> 0
        Return a list of prediction of new observations.
        threshold must be in (0,1)
        """

        if threshold >= 1 or threshold <= 0:
            raise ValueError(" threshold value must be in (0,1)")

        w = self.get()[0]
        try:
            X_test.shape[1]
        except:
            X_test = X_test.reshape(X_test.shape[0], 1)

        if self.preprocessing == 'Standardization':
            for i in range(X_test.shape[1]):
                X_test[:, i] = self._Standardization(X_test[:, i])

        elif self.preprocessing == 'Normalization':
            for i in range(X_test.shape[1]):
                X_test[:, i] = self._Normalization(X_test[:, i])
        elif self.preprocessing == 'None':
            X_test = X_test

        ones = np.ones((X_test.shape[0], 1))
        X_test = np.concatenate((ones, X_test), axis=1)

        y_test_hat = self._sigmoid(np.dot(X_test, w))
        label_pred = np.array(
            [0 if i < threshold else 1 for i in y_test_hat])

        return label_pred

# 3a: Tạo dữ liệu

In [ ]:
X, y = make_classification(n_features=2, n_classes=2,
                           n_redundant=0, n_samples=200, random_state=42)

In [ ]:
feature_1 = X.T[0]
feature_2 = X.T[1]
df = pd.DataFrame({'feature_1': feature_1,
                   'feature_2': feature_2,
                   'y': y})

df

#3b: Visualize dữ liệu

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.scatter_3d(data_frame=df, x='feature_1', y='feature_2',
                    z='y', color=y,  color_continuous_scale='Inferno')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.title("Data visualization")
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

# 3c: Chia dữ liệu thành train và test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# 4: Fit dữ liệu với Logistic Regression

In [ ]:
w_init = np.zeros((3, 1))
myLr = MyLogisticRegression(w_init, max_iteration=100)

In [ ]:
myLr.preprocessing(X_train, y_train, preprocessing='None')

In [ ]:
print(f'estimated coefficients are', myLr.get()[0])

In [ ]:
print(f'Prediction on test set \n', myLr.predict(X_test))

In [ ]:
# when threshold < 0 and  > 1, raise error
myLr.predict(X_test, thresh_hold=1)

### So sánh với sklearn


In [ ]:
from sklearn.linear_model import LogisticRegression

lrSK = LogisticRegression(penalty='none', solver='newton-cg')
lrSK.fit(X_train, y_train)
print(f'The intercept is', lrSK.intercept_)
print(f'The slopes are: ', lrSK.coef_)


print(f'Prediction by Sklearn: \n', lrSK.predict(X_test))

print(f'Compare results of sklearn vs myLr: \n',  myLr.predict(
    X_test, threshold=0.5) == lrSK.predict(X_test))

# 4b: Tính f1-score

In [ ]:
y_pred_on_train_manual = myLr.predict(X_train)
y_pred_on_test_manual = myLr.predict(X_test)

print(f'F1 Score on training set: ', F1_score(y_actual=y_train.reshape(1, -1), y_pred=y_pred_on_train_manual))

print(f'F1 Score on test set: ', F1_score(y_actual = y_test.reshape(1, -1), y_pred= y_pred_on_test_manual))

In [ ]:
from sklearn.metrics import f1_score
y_pred_on_train_sklearn = lrSK.predict(X_train)
y_pred_on_test_sklearn =lrSK.predict(X_test)


f1_on_train_by_sklearn = f1_score(y_true=y_train, y_pred=y_pred_on_train_sklearn)
f1_on_test_by_sklearn = f1_score(y_true=y_test, y_pred=y_pred_on_test_sklearn)


print(f'F1 Score on training set by Sklearn: ', f1_on_train_by_sklearn)
print(f'F1 Score on test set by Sklearn: ', f1_on_test_by_sklearn)


# 4c: Visualize loss

In [ ]:
all_loss = myLr.get()[-1]

x  = list(range(0,len(all_loss)))

plt.scatter(x, all_loss, s = 11, label = 'loss')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.legend()
plt.show()



In [ ]:
# # @title # 5c Visualize loss với learning rate và iter khác nhau {"display-mode": "form"}
# # Các bạn có thể thêm param "run": "auto" vào trong dictionary để cell tự động chạy khi bạn chỉnh param
# lr = 0.086  # @param {type:"slider", min:0, max:1, step:0.0005}
# iter = 977  # @param {type:"slider", min:0, max:1000, step:1}

# # Phần trên dành cho các bạn sử dụng Google Colab, với các bạn sử dụng Jupyter , các bạn hãy xoá tất cả và chạy code trong ô này nhé!
# ...